In [1]:
import sys,os
path = os.path.dirname(os.path.dirname("/home/jrj/postgraduate/vpg/prepare/dataset.ipynb"))
sys.path.append(path)
from utils.sol_utils import all_path, get_solc_version, is_mapping_type, is_number_type
from tqdm import tqdm
from prepare.graph import SolidityInfo
from thefuzz import fuzz

In [6]:
files = all_path("/home/jrj/postgraduate/Ethereum_smart_contract_datast/Ethereum_smart_contract_datast/contract_dataset_ethereum/", [".sol"])
success_files = []
error_files = []
mapping_vars_dict = dict()

In [8]:
print(f"success files:{len(success_files)}")
print(f"error files:{len(error_files)}")
print(f"success percentage:{len(success_files)/len(files)}")

success files:29956
error files:10793
success percentage:0.7351346045301725


In [3]:
import pickle
with open("../data/graph_v3/success_files.pkl", "wb") as fw:
    pickle.dump(success_files, fw)
with open("../data/graph_v3/error_files.pkl", "wb") as fw:
    pickle.dump(error_files, fw)
with open("../data/graph_v3/mapping_vars_dict.pkl", "wb") as fw:
    pickle.dump(mapping_vars_dict, fw)

NameError: name 'success_files' is not defined

In [2]:
import pickle
with open("../data/graph_v2/mapping_vars_dict.pkl", "rb") as fr:
    mapping_vars_dict = pickle.load(fr)
with open("../data/graph_v2/error_files.pkl", "rb") as fr:
    error_files = pickle.load(fr)
with open("../data/graph_v2/success_files.pkl", "rb") as fr:
    success_files = pickle.load(fr)

In [9]:
num_mapping = 0
num_true = 0
num_false = 0
special_list = []
for file in mapping_vars_dict:
    var_list = mapping_vars_dict[file]
    if len(var_list) >2:
        special_list.append(file)
    num_mapping += len(var_list)
    for var in var_list:
        if var[-1] == 1:
            num_true += 1
        else:
            num_false += 1
print(f"total:{num_mapping}")
print(f"num_true:{num_true/num_mapping}")
print(f"num_false:{num_false/num_mapping}")

total:48759
num_true:0.4208453823909432
num_false:0.5791546176090568


In [2]:
error_vpg_list = []

In [3]:
with open("../data/graph_v2/graph_data_0.pkl", "rb") as fr:
    graph_data_0 = pickle.load(fr)

In [4]:
from prepare.graph import VPG
graph_data = {}
label_data = {}
files = list(mapping_vars_dict.keys())
batch_size = 1000
for index in tqdm(range(len(files))):
    if index % batch_size == 0:
        graph_data = {}
        label_data = {}
    file = files[index]
    mapping_vars_name = mapping_vars_dict[file]
    mapping_vars_node = []
    mapping_vars_label = {}
    if len(mapping_vars_dict[file]) == 0: continue
    try:
        solc_version = get_solc_version(file)
        solc_version = f'/home/jrj/.solc-select/artifacts/solc-{solc_version}/solc-{solc_version}'
        sol = SolidityInfo(file, solc=solc_version)
        top_contracts = sol.top_contracts
        for tc in top_contracts:
            mapping_vars_node += list(filter(lambda x: is_mapping_type(x.type), tc.state_variables))
        graph = VPG(sol)
        for name, label in mapping_vars_name:
            for node in mapping_vars_node:
                if name == node.canonical_name and node in graph.nodes_index:
                    mapping_vars_label[graph.nodes_index[node][0]] = label
                    break
        label_data[file] = mapping_vars_label
        graph_data[file] = {
            "nodes_index": list(graph.nodes_index.values()),
            "edges": graph.edges
        }
        if index % batch_size == 0:
            with open(f"../data/graph_data_{str(index/batch_size)}.pkl", "wb") as fw:
                pickle.dump(graph_data, fw)
            with open(f"../data/label_data_{str(index/batch_size)}.pkl", "wb") as fw:
                pickle.dump(graph_data, fw)
    except Exception as e:
        error_vpg_list.append(file)

 57%|█████▋    | 16195/28179 [8:58:43<18:12:35,  5.47s/it] 

In [8]:
import pickle
from prepare.graph import VPG
with open("../data/mapping_vars_dict.pkl", "rb") as fr:
    mapping_vars_dict = pickle.load(fr)
files = list(mapping_vars_dict)
index = 1
batch_size = 1000
graph_data = {}
label_data = {}
for file in tqdm(files[index * batch_size:(index + 1) * batch_size]):
    mapping_vars_name = mapping_vars_dict[file]
    mapping_vars_node = []
    mapping_vars_label = {}
    if len(mapping_vars_dict[file]) == 0: continue
    try:
        solc_version = get_solc_version(file)
        solc_version = f'/home/jrj/.solc-select/artifacts/solc-{solc_version}/solc-{solc_version}'
        sol = SolidityInfo(file, solc=solc_version)
        top_contracts = sol.top_contracts
        for tc in top_contracts:
            mapping_vars_node += list(filter(lambda x: is_mapping_type(x.type), tc.state_variables))
        graph = VPG(sol)
        for name, label in mapping_vars_name:
            for node in mapping_vars_node:
                if name == node.canonical_name and node in graph.nodes_index:
                    mapping_vars_label[graph.nodes_index[node][0]] = label
                    break
        label_data[file] = mapping_vars_label
        graph_data[file] = {
            "nodes_index": list(graph.nodes_index.values()),
            "edges": list(graph.edges)
        }
    except Exception as e:
        print(e)
        error_vpg_list.append(file)

100%|██████████| 1000/1000 [12:51<00:00,  1.30it/s] 


In [3]:
mapping_vars_dict['/home/jrj/postgraduate/Ethereum_smart_contract_datast/Ethereum_smart_contract_datast/contract_dataset_ethereum/contract37/36745.sol']

[['ERC223BasicToken.balances', 1], ['StandardToken.allowed', 0]]

In [13]:
g = graph_data["/home/jrj/postgraduate/Ethereum_smart_contract_datast/Ethereum_smart_contract_datast/contract_dataset_ethereum/contract17/16244.sol"]

In [1]:
import pickle
with open("/home/jrj/postgraduate/vpg/data/graph_data_1.pkl","rb") as fr:
    a = pickle.load(fr)

In [1]:
from torch import nn
import torch

In [3]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 2, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(2)


In [4]:
output = loss(input, target)

In [5]:
output.backward()

In [8]:
input = torch.randn(3, 2, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(2)
output = loss(input, target)

In [4]:
loss = nn.BCEWithLogitsLoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(input, target)